In [1]:
from utilities.utils import generate_dates,download_ionex_v2,date_to_ionex_name,download_sp3_v2,download_clk,ION_root,SP3_root,CLK_root,RNX_root,TEMP_root,download_rinex,prioritized_sp3_filenames,date_to_rinex_name
import numpy as np
import glob
import os
import pickle
import datetime

rinex format: https://files.igs.org/pub/data/format/

notes:https://destevez.net/2023/09/galileo-gst-utc-anomaly/

Solution types: https://files.igs.org/pub/resource/guidelines/Guidelines_For_Long_Product_Filenames_in_the_IGS_v2.0.pdf (page 6)

2.2 Solution Type Identifiers
 The following solution type identifiers (TTT) are currently supported:
1. FIN Final products*
2. NRT Near-Real Time products (products between ULT and RTS)
3. PRD Predicted products
4. RAP Rapid products*
5. RTS Real-Time streamed products
6. SNX SINEX Combination product
7. ULT Ultra-rapid products (every 6 hours)
* Definitions as agreed by the relevant IGS Working Group

#### Generate dates or load file

In [2]:
os.makedirs(TEMP_root,exist_ok=True)

dates_set_file_path = os.path.join(TEMP_root,'dates_set_single.pk')
stations_file_path =  os.path.join(TEMP_root,'stations_single.pk')
agencies_file_path =  os.path.join(TEMP_root,'agencies.pk')

# years_list = [2012,2013,2015,2016,2020,2021]+[2014]
# years_list = [2022,2023,2024]
years_list = [2025]

if os.path.isfile(agencies_file_path):
    agencies = pickle.load(open(agencies_file_path,'rb'))
else:
    agencies = ['upc','ckm']
    pickle.dump(agencies, open(agencies_file_path,'wb'))

if os.path.isfile(stations_file_path):
    stations = pickle.load(open(stations_file_path,'rb'))
else:
    stations = ['ramo']
    # stations = ['alx2','phlw','tays','rash','haly','elat','nrif','petah','spir','ramo','nizn','yrcm','dsea','muta','drag','alon','jslm','ammn','hugs','tela','mrav','just','csar','kabr','katz','hram','udmc','laug','hrrn','zako','iser','izqw','albr','issd','izad','izbl','isku','isna','hamd','tehn','abrk','shrz','ahvz','isbs','kuwt']
    pickle.dump(stations, open(stations_file_path,'wb'))



if os.path.isfile(dates_set_file_path):
    dates_sets = pickle.load(open(dates_set_file_path,'rb'))
else:
    random_seed=223
    np.random.seed(random_seed)
    dates_sets = []
    for year in years_list:
        # dates_sets += generate_dates(year,n_contious_dates=365,n_generations=1)# + datetime.timedelta(days=31)
        dates_sets += generate_dates(year,n_contious_dates=1,n_generations=1,up_to=datetime.datetime(2025,3,1))# + datetime.timedelta(days=31)
    # dates_sets += generate_dates(2014,n_contious_dates=365,n_generations=1)
    dates_sets.sort()

    pickle.dump(dates_sets, open(dates_set_file_path,'wb'))

dates_sets,agencies,stations

([[datetime.datetime(2025, 1, 31, 0, 0)]], ['upc', 'ckm'], ['ramo'])

In [3]:

# file_type = RNX_root

# agency = 'ckm'

def download_ION_SP3_CLK(agency,file_type,dates_sets):

    _names = [date_to_ionex_name(d,agency)[0] for w in dates_sets for d in w]
    _in_folder = glob.glob("./{}/*i".format(file_type))
    _in_folder = [n.split("/")[-1] for n in _in_folder]
    max_retries = 1

    while len(set(_names)-set(_in_folder)) > 0 and max_retries > 0:
        for i in range(len(dates_sets)):

            seq = dates_sets[i]

            try:
                if file_type == ION_root:
                    print(agency,file_type)
                    # download_ionex(seq)
                    download_ionex_v2(seq)
                elif file_type == SP3_root:
                    print(agency,file_type)
                    download_sp3_v2(seq)
                elif file_type == CLK_root:
                    print(agency,file_type)
                    download_clk(seq)
            except:
                i -= 1
        max_retries -= 1


def download_RNX(stations,dates_sets):
    for station in stations:
        for seq in dates_sets:
            # print(seq)
            download_rinex(station,seq)


for file_type in [ION_root,SP3_root,CLK_root]:
    for agency in agencies:
        # print(agency,file_type)

        download_ION_SP3_CLK(agency,file_type,dates_sets)


upc ION
upc SP3
ckm SP3
upc CLK
ckm CLK


In [4]:
# download_RNX(stations,dates_sets)